# FAIR Jupyter
FAIR Jupyter is a knowledge graph for semantic sharing and granular exploration of a computational notebook reproducibility dataset. This notebook provides some SPARQL queries to query the FAIR Jupyter SPARQL Endpoint.

More Information on FAIR Jupyter Ontology and Knowledge Graph: https://w3id.org/fairjupyter

## Prerequisites
The notebook is written in Python and besides Jupyter and Wikidata, it has the following direct dependencies:
* sparqlwrapper to run SPARQL queries on Wikidata's SPARQL endpoint
* pandas to output the query results in spreadsheet format

In [16]:
!pip install sparqlwrapper

In [17]:
!pip install pandas

In [18]:
import csv
import os
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML

# Functions
To achieve consistent output across multiple queries, the queries call a custom function select that queries FAIR Jupyter SPARQL Endpoint and saves the query results in the spreadsheet format CSV.

In [19]:
sparql_endpoint = 'https://reproduceme.uni-jena.de/fairjupyter/sparql'
def select(query, service=sparql_endpoint):    
    sparql = SPARQLWrapper(service)    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)        
    result = sparql.query().convert()   
    return result

In [20]:
def query_and_display_results(query, csv_filename):
    # Create a "results" folder if it doesn't exist
    results_folder = "results"
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)
        
    # Perform the SPARQL query
    results = select(query)
    
    # Extract the headers from the results
    headers = results["head"]["vars"]
    
    # Construct the full CSV file path inside the results folder
    csv_file = os.path.join(results_folder, csv_filename + '.csv')
    
    # Initialize a list to store the rows for returning later
    result_rows = []
    
    # Open CSV file and write the results
    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        
        # Write the headers
        writer.writerow(headers)
        
        # Write the rows from the SPARQL query results and store in result_rows
        for result in results["results"]["bindings"]:
            row = [result[var]["value"] if var in result else "" for var in headers]
            writer.writerow(row)
            result_rows.append(row)

    print(f"Results written to {csv_file}")
    
    # Return the results as a DataFrame
    df_results = pd.DataFrame(result_rows, columns=headers)    
    return df_results


# SPARQL queries to the KG that reproduce materials from the original manuscript describing the dataset

## Research articles by research field

In [21]:
query_string = """
SELECT ?research_field (COUNT(DISTINCT ?article) AS ?number_of_articles)
WHERE {  
  ?repository <http://purl.org/pav/retrievedFrom> ?article .
  ?article <http://www.w3.org/ns/prov-o#specializationOf> ?mesh .
  ?mesh <http://www.w3.org/ns/prov-o#generalizationOf> ?top_mesh .
  ?top_mesh <http://www.w3.org/2000/01/rdf-schema#label> ?research_field
  
}
GROUP BY ?research_field
ORDER BY DESC(?number_of_articles)
LIMIT 10

"""
csv_filename = 'fig_3_research_field_by_no_of_articles'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fig_3_research_field_by_no_of_articles.csv


,research_field,number_of_articles
0,Eukaryota,1153
1,Information Science,1024
2,Investigative Techniques,989
3,Genetic Phenomena,641
4,Natural Science Disciplines,539
5,Environment and Public Health,507
6,Mathematical Concepts,467
7,"Amino Acids, Peptides, and Proteins",446
8,Chemical Phenomena,421
9,"Health Care Quality, Access, and Evaluation",361


## Research field (MeSH terms) by the number of GitHub repositories  that contain at least one Jupyter notebook.

In [22]:
query_string = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX repr: <https://w3id.org/reproduceme/>

SELECT ?research_field (COUNT(DISTINCT ?repository) as ?repository_count) (COUNT(DISTINCT ?repository_nb) as ?repositories_with_notebooks_count)
WHERE {
  {
  ?repository <http://purl.org/pav/retrievedFrom> ?article .
  }
  UNION
  {
  ?repository_nb <http://purl.org/pav/retrievedFrom>  ?article ;
              repr:notebooks_count ?notebooks_count .
  	FILTER(xsd:integer(?notebooks_count) > 0)
  }
  ?article <http://www.w3.org/ns/prov-o#specializationOf> ?mesh .
  ?mesh <http://www.w3.org/ns/prov-o#generalizationOf> ?top_mesh .
  ?top_mesh rdfs:label ?research_field
}
GROUP BY ?research_field
ORDER BY DESC(?repository_count)
LIMIT 10
"""
csv_filename = 'fig_4_research_field_by_no_of_repos'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fig_4_research_field_by_no_of_repos.csv


,research_field,repository_count,repositories_with_notebooks_count
0,Eukaryota,2375,1207
1,Information Science,2344,1104
2,Investigative Techniques,2076,1048
3,Genetic Phenomena,1271,680
4,Natural Science Disciplines,1193,556
5,Environment and Public Health,1038,473
6,Mathematical Concepts,1017,530
7,Chemical Phenomena,813,467
8,"Amino Acids, Peptides, and Proteins",783,448
9,"Health Care Quality, Access, and Evaluation",735,343


## Journals with the highest number of articles that had a valid GitHub repository and at least one Jupyter notebook.

In [23]:
query_string = """
SELECT ?journal_name (COUNT(?article) as ?article_count)
WHERE {
  ?article <https://w3id.org/reproduceme/publishedIn> ?journal .
  ?journal <http://www.w3.org/2000/01/rdf-schema#label> ?journal_name .
}
GROUP BY ?journal_name
ORDER BY DESC(?article_count)
LIMIT 10
"""
csv_filename = 'fig_5_journals_articles_valid_repo_notebook'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fig_5_journals_articles_valid_repo_notebook.csv


,journal_name,article_count
0,Nature Communications,178
1,eLife,151
2,Scientific Reports,107
3,PLoS Computational Biology,101
4,Scientific Data,100
5,PLoS ONE,96
6,Bioinformatics,91
7,BMC Bioinformatics,67
8,Frontiers in Neuroinformatics,51
9,"Sensors (Basel, Switzerland)",51


## Journals by the number of GitHub repositories and by the number of GitHub repositories with at least one Jupyter notebook.

In [24]:
query_string = """
SELECT ?journal_name (COUNT(?repository) as ?repository_count) (COUNT(?repository_nb) as ?repositories_with_notebooks_count) WHERE
  {
  ?article <https://w3id.org/reproduceme/publishedIn> ?journal .
  ?journal <http://www.w3.org/2000/01/rdf-schema#label> ?journal_name .
  {
  ?repository <http://purl.org/pav/retrievedFrom> ?article .
  }
  UNION
  {
  ?repository_nb <http://purl.org/pav/retrievedFrom> ?article ;
              <https://w3id.org/reproduceme/notebooks_count> ?notebooks_count .
  FILTER(<http://www.w3.org/2001/XMLSchema#integer>(?notebooks_count) > 0)
  }
  }
  GROUP BY ?journal_name

ORDER BY DESC(?repository_count)
LIMIT 10
"""
csv_filename = 'fig_6_journals_by_repo_notebook'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fig_6_journals_by_repo_notebook.csv


,journal_name,repository_count,repositories_with_notebooks_count
0,eLife,384,212
1,Nature Communications,336,169
2,PLoS Computational Biology,187,95
3,Scientific Data,186,111
4,PLoS ONE,161,93
5,BMC Bioinformatics,153,65
6,GigaScience,139,47
7,Scientific Reports,136,89
8,Bioinformatics,135,71
9,Frontiers in Neuroinformatics,114,42


## Journals by number of GitHub repositories with Jupyter notebooks.

In [25]:
query_string = """
SELECT ?journal_name (COUNT(?repository_nb) AS ?repositories_with_notebooks_count)
        ?max_notebooks_count
WHERE {
  {
    SELECT ?journal (MAX(?notebooks_count) AS ?max_notebooks_count)
    WHERE {
      ?article <https://w3id.org/reproduceme/publishedIn> ?journal .
      ?journal <http://www.w3.org/2000/01/rdf-schema#label> ?journal_name .  
      ?repository_nb <http://purl.org/pav/retrievedFrom> ?article ;
                     <https://w3id.org/reproduceme/notebooks_count> ?notebooks_count .
      FILTER(<http://www.w3.org/2001/XMLSchema#integer>(?notebooks_count) > 0)  
    }
    GROUP BY ?journal
  }
  ?article <https://w3id.org/reproduceme/publishedIn> ?journal .
  ?journal <http://www.w3.org/2000/01/rdf-schema#label> ?journal_name .  
  ?repository_nb <http://purl.org/pav/retrievedFrom> ?article ;
                 <https://w3id.org/reproduceme/notebooks_count> ?notebooks_count .
  FILTER(<http://www.w3.org/2001/XMLSchema#integer>(?notebooks_count) > 0)  
}
GROUP BY ?journal_name  ?max_notebooks_count
ORDER BY DESC(?repositories_with_notebooks_count)
LIMIT 10

"""
csv_filename = 'fig_7_journals_by_repo_with_notebooks'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fig_7_journals_by_repo_with_notebooks.csv


,journal_name,repositories_with_notebooks_count,max_notebooks_count
0,eLife,212,91
1,Nature Communications,169,9
2,Scientific Data,111,92
3,PLoS Computational Biology,95,9
4,PLoS ONE,93,9
5,Scientific Reports,89,9
6,Bioinformatics,71,9
7,BMC Bioinformatics,65,9
8,GigaScience,47,9
9,Frontiers in Neuroinformatics,42,9


## Programming languages of the notebooks.

In [26]:
query_string = """
SELECT ?language (COUNT(?notebook) as ?notebook_count)
WHERE {
  ?notebook a <https://w3id.org/reproduceme/Notebook> ;
            <https://w3id.org/reproduceme/language> ?language .
}
GROUP BY ?language
ORDER BY DESC(?notebook_count)
LIMIT 10
"""
csv_filename = 'fig_9_programming_languages'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fig_9_programming_languages.csv


,language,notebook_count
0,python,22578
1,unknown,3112
2,R,891
3,julia,295
4,matlab,134
5,Groovy,95
6,bash,36
7,Scala,29
8,sos,29
9,Java,8


## Relative proportion of the most frequent programming languages used in the notebooks per year.

In [ ]:
query_string = """
SELECT ?created_year ?language (COUNT(?notebook) as ?notebook_count)
WHERE {
  ?notebook a <https://w3id.org/reproduceme/Notebook> ;
            <http://purl.org/pav/retrievedFrom>  ?repository ;
            <https://w3id.org/reproduceme/language> ?language ;
            <https://w3id.org/reproduceme/language_version> ?version .
  ?repository <https://w3id.org/reproduceme/created_at> ?created_at .
  BIND(REPLACE(str(?created_at), "(\\d*)-.*", "$1") AS ?created_year)  
}
GROUP BY ?created_year ?language
ORDER BY ?created_year ?language


"""
csv_filename = 'fig_10_top_programming_languages_per_year'
results = query_and_display_results(query_string, csv_filename)
results

## Python notebooks by minor Python version by year of last commit to the GitHub repository containing the notebook.

In [ ]:
query_string = """
SELECT ?created_year ?minor_version (COUNT(?notebook) as ?count_minor_version)
WHERE {
  ?notebook a <https://w3id.org/reproduceme/Notebook> ;
            <http://purl.org/pav/retrievedFrom>  ?repository ;
            <https://w3id.org/reproduceme/language> "python" ;
            <https://w3id.org/reproduceme/language_version> ?version .
  ?repository <https://w3id.org/reproduceme/created_at> ?created_at .
  BIND(REPLACE(str(?created_at), "(\\d*)-.*", "$1") AS ?created_year)  
  BIND(SUBSTR(?version, 1, 3) AS ?minor_version)
  FILTER(?version != "3" && ?version != "1" && ?version != "ES2015")
}
GROUP BY ?created_year ?minor_version
ORDER BY ?created_year ?minor_version


"""
csv_filename = 'fig_11_python_notebooks_by_minor_version_last_commit'
results = query_and_display_results(query_string, csv_filename)
results

## Python notebooks by major Python version by year of first commit to the notebook’s GitHub repository.

In [ ]:
query_string = """
SELECT ?created_year ?major_version (COUNT(?notebook) as ?count_major_version)
WHERE {
  ?notebook a <https://w3id.org/reproduceme/Notebook> ;
            <http://purl.org/pav/retrievedFrom>  ?repository ;
            <https://w3id.org/reproduceme/language> "python" ;
            <https://w3id.org/reproduceme/language_version> ?version .
  ?repository <https://w3id.org/reproduceme/created_at> ?created_at .
  BIND(REPLACE(str(?created_at), "(\\d*)-.*", "$1") AS ?created_year)  
  BIND(SUBSTR(?version, 1, 1) AS ?major_version)
  FILTER(?version != "3" && ?version != "1" && ?version != "ES2015")
}
GROUP BY ?created_year ?major_version
ORDER BY ?created_year ?major_version


"""
csv_filename = 'fig_12_python_notebooks_by_major_version_first_commit'
results = query_and_display_results(query_string, csv_filename)
results

## Exceptions occurring in Jupyter notebooks in our corpus.

In [31]:
query_string = """
SELECT ?exception (COUNT(?exception) AS ?count)
WHERE {
  ?execution  a <https://w3id.org/reproduceme/CellExecution> ;
    <https://w3id.org/reproduceme/exception> ?exception .
}
GROUP BY ?exception
ORDER BY DESC(?count)
LIMIT 10

"""
csv_filename = 'fig_19_top_exceptions'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fig_19_top_exceptions.csv


,exception,count
0,ModuleNotFoundError,5562
1,FileNotFoundError,1102
2,ImportError,1026
3,NameError,390
4,IOError,147
5,AttributeError,94
6,ValueError,91
7,TypeError,85
8,KeyError,71
9,CalledProcessError,68


## Jupyter notebook exceptions by research field, taking as a proxy the highest-level MeSH terms of the article associated with the notebook.

In [32]:
query_string = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?research_field (COUNT(?exception) AS ?exception_count)
WHERE {  
  ?execution  a <https://w3id.org/reproduceme/CellExecution> ;
    <https://w3id.org/reproduceme/exception> ?exception ;
    <http://purl.org/pav/retrievedFrom> ?repository .
  ?repository a <http://usefulinc.com/ns/doap#GitRepository> ;
  			<http://purl.org/pav/retrievedFrom> ?article ;
  			<https://w3id.org/reproduceme/notebooks_count> ?notebooks_count .
  ?article a <http://purl.org/spar/fabio/Article> ; 
  		 <http://www.w3.org/ns/prov-o#specializationOf> ?mesh .
  ?mesh <http://www.w3.org/ns/prov-o#generalizationOf> ?top_mesh .
  ?top_mesh <http://www.w3.org/2000/01/rdf-schema#label> ?research_field .    
  FILTER (xsd:integer(?notebooks_count)>0)
}
GROUP BY ?research_field
ORDER BY DESC(?exception_count)
LIMIT 10

"""
csv_filename = 'fig_22_exceptions_by_research_field'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fig_22_exceptions_by_research_field.csv


,research_field,exception_count
0,Genetic Phenomena,13309
1,Investigative Techniques,12819
2,Eukaryota,11327
3,Information Science,10169
4,Nervous System,10068
5,Chemical Phenomena,6049
6,Cells,5969
7,"Amino Acids, Peptides, and Proteins",5008
8,Environment and Public Health,3826
9,Natural Science Disciplines,3663


## Notebooks with successful executions with same and different results

In [33]:
query_string = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT (COUNT(?processed_different_result) AS ?count_different_result) (COUNT(?processed_same_result) AS ?count_same_result) (?count_same_result + ?count_different_result AS ?count_successful_executions)
WHERE {
  ?execution a <https://w3id.org/reproduceme/CellExecution> .
  OPTIONAL { ?execution <https://w3id.org/reproduceme/exception> ?exception . }
  OPTIONAL {
    ?execution <https://w3id.org/reproduceme/processed> ?processed_different_result .
    FILTER ((xsd:integer(?processed_different_result) = 35) && !bound(?exception))
  }
  OPTIONAL {
    ?execution <https://w3id.org/reproduceme/processed> ?processed_same_result .
    FILTER ((xsd:integer(?processed_same_result) = 51) && !bound(?exception))
  }
  
  
}
"""
csv_filename = 'table_2_successful_executions_with_same_different_results'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/table_2_successful_executions_with_same_different_results.csv


,count_different_result,count_same_result,count_successful_executions
0,324,879,1203


## Common Python code warnings/ style errors in our notebook corpus.

In [34]:
query_string = """
SELECT ?notebook ?error ?description
WHERE {
  ?error a <https://w3id.org/reproduceme/NotebookCodeStyleError> ;
        <https://w3id.org/reproduceme/description> ?description ;
        <http://purl.org/pav/retrievedFrom> ?notebook .
}
LIMIT 10
"""
csv_filename = 'table_4_code_styling_errors'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/table_4_code_styling_errors.csv


,notebook,error,description
0,https://w3id.org/reproduceme/notebook_24829,https://w3id.org/reproduceme/E711,comparison to None should be 'if cond is None:'
1,https://w3id.org/reproduceme/notebook_6361,https://w3id.org/reproduceme/E711,comparison to None should be 'if cond is None:'
2,https://w3id.org/reproduceme/notebook_16999,https://w3id.org/reproduceme/E711,comparison to None should be 'if cond is None:'
3,https://w3id.org/reproduceme/notebook_24760,https://w3id.org/reproduceme/E711,comparison to None should be 'if cond is None:'
4,https://w3id.org/reproduceme/notebook_24825,https://w3id.org/reproduceme/E711,comparison to None should be 'if cond is None:'
5,https://w3id.org/reproduceme/notebook_26121,https://w3id.org/reproduceme/E711,comparison to None should be 'if cond is None:'
6,https://w3id.org/reproduceme/notebook_1386,https://w3id.org/reproduceme/E711,comparison to None should be 'if cond is None:'
7,https://w3id.org/reproduceme/notebook_24874,https://w3id.org/reproduceme/E711,comparison to None should be 'if cond is None:'
8,https://w3id.org/reproduceme/notebook_16823,https://w3id.org/reproduceme/E711,comparison to None should be 'if cond is None:'
9,https://w3id.org/reproduceme/notebook_19134,https://w3id.org/reproduceme/E711,comparison to None should be 'if cond is None:'


# Other queries over the FAIR Jupyter graph

## Notebooks by search term: 'immun' AND ('stem' OR 'differentiation')

In [35]:
query_string = """
SELECT DISTINCT ?notebook_url ?article_label ?keywords WHERE { 
  ?article <https://w3id.org/reproduceme/keywords> ?keywords .
  ?article <http://www.w3.org/2000/01/rdf-schema#label> ?article_label . 
  ?article <https://w3id.org/reproduceme/publishedIn> ?journal .
  ?journal <http://www.w3.org/2000/01/rdf-schema#label> ?journal_label . 
  FILTER (REGEX(LCASE(CONCAT(?keywords, " ", ?article_label, " ", ?journal_label)), "immun"))
  FILTER (REGEX(LCASE(CONCAT(?keywords, " ", ?article_label, " ", ?journal_label)), "\\b(stem|differentiation)"))
  ?article ^<http://purl.org/pav/retrievedFrom> ?repository .
  ?notebook <http://purl.org/pav/retrievedFrom> ?repository .
  ?notebook <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://w3id.org/reproduceme/Notebook> .
  ?notebook <http://www.w3.org/2000/01/rdf-schema#label> ?notebook_label . # filename
  ?repository <https://w3id.org/reproduceme/url> ?repo_url_base . # find repo on GitHub
  BIND(URI(CONCAT( ?repo_url_base, "/blob/master/", ?notebook_label)) AS ?notebook_url) # create clickable link to notebook on GitHub
  FILTER (?notebook_url != "")
}

"""
csv_filename = 'notebooks:by_search_term'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/notebooks:by_search_term.csv


,notebook_url,article_label,keywords


## Article by keywords, e.g., `open source'

In [36]:
query_string = """
SELECT DISTINCT ?article ?keywords WHERE { 
  ?article <https://w3id.org/reproduceme/keywords> ?keywords .
  FILTER (REGEX(LCASE(?keywords), "open(.)source"))
}

"""
csv_filename = 'articles_by_keywords_open_source'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/articles_by_keywords_open_source.csv


,article,keywords
0,https://w3id.org/reproduceme/article_2442,sigma profile;vapor-liquid-equilibria;COSMO-SA...
1,https://w3id.org/reproduceme/article_2376,open‐source software data analysis tool based ...
2,https://w3id.org/reproduceme/article_2567,Metabolomics;Partial least squares;Support vec...
3,https://w3id.org/reproduceme/article_29,Gene regulation;Multi-omic analysis;Network bi...
4,https://w3id.org/reproduceme/article_798,auditory feedback;tapping;social;individual di...
5,https://w3id.org/reproduceme/article_2934,education;software;informatics;bioinformatics;...
6,https://w3id.org/reproduceme/article_2784,Computer-aided drug design;Python;RDKit;Open s...
7,https://w3id.org/reproduceme/article_1887,spectral smile;optical aberration;aberration c...
8,https://w3id.org/reproduceme/article_769,data ethics;responsible innovation;journal clu...
9,https://w3id.org/reproduceme/article_1962,Brain–Machine Interfaces;Bioimaging;Deep learn...


## Most common errors in immunology

In [37]:
query_string = """
SELECT DISTINCT ?exception (COUNT(?exception) AS ?count) WHERE { 
    ?execution  a <https://w3id.org/reproduceme/CellExecution> ;
    <https://w3id.org/reproduceme/exception> ?exception ;
    <http://purl.org/pav/retrievedFrom> ?repository .
    ?repository a <http://usefulinc.com/ns/doap#GitRepository> ;
  			<http://purl.org/pav/retrievedFrom> ?article .
  ?article  <https://w3id.org/reproduceme/keywords> ?keywords .
  FILTER (REGEX(LCASE(?keywords), "immun"))
}
GROUP BY ?exception
ORDER BY DESC(?count)

"""
csv_filename = 'top_errors_immunology'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/top_errors_immunology.csv


,exception,count
0,ModuleNotFoundError,51
1,FileNotFoundError,24
2,ImportError,16
3,CalledProcessError,7
4,NameError,6
5,AttributeError,1
6,RuntimeError,1


## Most common errors in Nature journal

In [38]:
query_string = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT  ?exception (COUNT(?exception) AS ?count) WHERE { 
    ?execution  a <https://w3id.org/reproduceme/CellExecution> ;
    <https://w3id.org/reproduceme/exception> ?exception ;
    <http://purl.org/pav/retrievedFrom> ?repository .
    ?repository a <http://usefulinc.com/ns/doap#GitRepository> ;
  			<http://purl.org/pav/retrievedFrom> ?article .
  ?article  <https://w3id.org/reproduceme/publishedIn> ?journal .
  ?journal rdfs:label ?journal_name
  FILTER (?journal_name="Nature")
}
GROUP BY ?exception
ORDER BY DESC(?count)

"""
csv_filename = 'top_errors_nature_journal'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/top_errors_nature_journal.csv


,exception,count
0,ModuleNotFoundError,1818
1,FileNotFoundError,67
2,AttributeError,34
3,<Unknown exception>,8
4,ImportError,8
5,ValueError,6
6,IndexError: too many indices for array,3
7,KeyError,2
8,RuntimeError,2
9,NameError,1


## MeSH terms ranked by 'ModuleNotFoundError' frequency

In [39]:
query_string = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?research_field (COUNT(?exception) AS ?exception_count)
WHERE {  
  ?execution  a <https://w3id.org/reproduceme/CellExecution> ;
    <https://w3id.org/reproduceme/exception> ?exception ;
    <http://purl.org/pav/retrievedFrom> ?repository .
  ?repository a <http://usefulinc.com/ns/doap#GitRepository> ;
  			<http://purl.org/pav/retrievedFrom> ?article ;
  			<https://w3id.org/reproduceme/notebooks_count> ?notebooks_count .
  ?article a <http://purl.org/spar/fabio/Article> ; 
  		 <http://www.w3.org/ns/prov-o#specializationOf> ?mesh .
  ?mesh <http://www.w3.org/ns/prov-o#generalizationOf> ?top_mesh .
  ?top_mesh <http://www.w3.org/2000/01/rdf-schema#label> ?research_field .    
  FILTER (?exception='ModuleNotFoundError')
}
GROUP BY ?research_field
ORDER BY DESC(?exception_count)
"""
csv_filename = 'exceptions_modulenotfounderror_by_researchfield'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/exceptions_modulenotfounderror_by_researchfield.csv


,research_field,exception_count
0,Investigative Techniques,9529
1,Nervous System,9070
2,Genetic Phenomena,9010
3,Eukaryota,8014
4,Information Science,6486
...,...,...
101,Urogenital System,2
102,Archaea,1
103,Musculoskeletal Diseases,1
104,Stomatognathic System,1


## Repositories by their stargazers count

In [40]:
query_string = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?repo ?stargazers_count WHERE {
  ?repo <https://w3id.org/reproduceme/stargazers_count> ?count. 
  BIND(xsd:float(?count) AS ?stargazers_count)
  FILTER ((?stargazers_count) > 0)
} 
ORDER BY DESC(?stargazers_count)
"""
csv_filename = 'repos_by_stargazers_count'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/repos_by_stargazers_count.csv


,repo,stargazers_count
0,https://w3id.org/reproduceme/repository_1785,174090
1,https://w3id.org/reproduceme/repository_4832,91488
2,https://w3id.org/reproduceme/repository_1365,75722
3,https://w3id.org/reproduceme/repository_541,75530
4,https://w3id.org/reproduceme/repository_4517,68653
...,...,...
4157,https://w3id.org/reproduceme/repository_927,1
4158,https://w3id.org/reproduceme/repository_956,1
4159,https://w3id.org/reproduceme/repository_957,1
4160,https://w3id.org/reproduceme/repository_974,1


# Federated queries between the FAIR Jupyter KG and Wikidata

## Match articles between FAIR Jupyter and Wikidata via DOI

In [41]:
query_string = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

PREFIX wikidata_wd: <http://www.wikidata.org/entity/>
PREFIX wikidata_wdt: <http://www.wikidata.org/prop/direct/>

SELECT DISTINCT

 ?fj_article
 ?wikidata
 ?wikidata_label
 ?DOI

WHERE {
  ?fj_article <https://w3id.org/reproduceme/doi> ?doi .
  BIND(UCASE(?doi) AS ?DOI)
  service <https://query.wikidata.org/sparql> {
    ?wikidata wikidata_wdt:P356 ?DOI .
    ?wikidata rdfs:label ?wikidata_label .
    FILTER (LANG(?wikidata_label) = "en")
  }
}
LIMIT 100
"""
csv_filename = 'fairjupyter_wikidata_match_articles_doi'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fairjupyter_wikidata_match_articles_doi.csv


,fj_article,wikidata,wikidata_label,DOI
0,https://w3id.org/reproduceme/article_1130,http://www.wikidata.org/entity/Q111150072,High-throughput qPCR and 16S rRNA gene amplico...,10.1186/S12866-022-02451-Y
1,https://w3id.org/reproduceme/article_1308,http://www.wikidata.org/entity/Q111149763,Accessible data curation and analytics for int...,10.1038/S41597-021-01071-X
2,https://w3id.org/reproduceme/article_2980,http://www.wikidata.org/entity/Q56394784,Digital open science-Teaching digital tools fo...,10.1371/JOURNAL.PBIO.2006022
3,https://w3id.org/reproduceme/article_1621,http://www.wikidata.org/entity/Q108126956,Analysis of single-cell RNA sequencing data ba...,10.1186/S12859-021-04150-3
4,https://w3id.org/reproduceme/article_2309,http://www.wikidata.org/entity/Q96174096,Perspective: Towards Automated Tracking of Con...,10.1093/ADVANCES/NMAA057
...,...,...,...,...
95,https://w3id.org/reproduceme/article_1508,http://www.wikidata.org/entity/Q108127212,Retrospective cell lineage reconstruction in h...,10.1016/J.CRMETH.2021.100054
96,https://w3id.org/reproduceme/article_646,http://www.wikidata.org/entity/Q125341804,Structure based design of effective HtpG-deriv...,10.3389/FMOLB.2022.964645
97,https://w3id.org/reproduceme/article_879,http://www.wikidata.org/entity/Q125345031,Complete Sequences of the Velvet Worm Slime Pr...,10.1002/ADVS.202201444
98,https://w3id.org/reproduceme/article_99,http://www.wikidata.org/entity/Q117886122,Insulin-like peptide 8 (Ilp8) regulates female...,10.3389/FCELL.2023.1103923


## Match articles between FAIR Jupyter and Wikidata via PMC ID

In [42]:
query_string = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

PREFIX wikidata_wd: <http://www.wikidata.org/entity/>
PREFIX wikidata_wdt: <http://www.wikidata.org/prop/direct/>

SELECT DISTINCT

 ?fj_article
 ?wikidata
 ?wikidata_label
 ?pmc

WHERE {
  ?fj_article <https://w3id.org/reproduceme/pmc> ?pmc .
  service <https://query.wikidata.org/sparql> {
    ?wikidata wikidata_wdt:P932 ?pmc .
    ?wikidata rdfs:label ?wikidata_label .
    FILTER (LANG(?wikidata_label) = "en")
  }
}
LIMIT 100
"""
csv_filename = 'fairjupyter_wikidata_match_articles_pmc_id'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fairjupyter_wikidata_match_articles_pmc_id.csv


,fj_article,wikidata,wikidata_label,pmc
0,https://w3id.org/reproduceme/article_3244,http://www.wikidata.org/entity/Q30491149,Meet Spinky: An Open-Source Spindle and K-Comp...,5332402
1,https://w3id.org/reproduceme/article_2712,http://www.wikidata.org/entity/Q93088604,A novel host-adapted strain of Salmonella Typh...,6597722
2,https://w3id.org/reproduceme/article_1956,http://www.wikidata.org/entity/Q108126755,Rapid discovery of novel prophages using biolo...,7787355
3,https://w3id.org/reproduceme/article_1150,http://www.wikidata.org/entity/Q111322094,Visualisation of proteome-wide ordered protein...,8889043
4,https://w3id.org/reproduceme/article_2860,http://www.wikidata.org/entity/Q61443014,"Calour: an Interactive, Microbe-Centric Analys...",6351725
...,...,...,...,...
95,https://w3id.org/reproduceme/article_2613,http://www.wikidata.org/entity/Q90413997,Estimating influenza incidence using search qu...,6771994
96,https://w3id.org/reproduceme/article_2432,http://www.wikidata.org/entity/Q90457588,CIViCpy: A Python Software Development and Ana...,7113080
97,https://w3id.org/reproduceme/article_1991,http://www.wikidata.org/entity/Q104484126,Using deep reinforcement learning to reveal ho...,7897245
98,https://w3id.org/reproduceme/article_2953,http://www.wikidata.org/entity/Q57074760,No Discrete Start/Stop Signals in the Dorsal S...,6231539


## Match articles between FAIR Jupyter and Wikidata via MeSH in different language, i.e Malayalam

In [43]:
query_string = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

PREFIX wikidata_wd: <http://www.wikidata.org/entity/>
PREFIX wikidata_wdt: <http://www.wikidata.org/prop/direct/>

SELECT DISTINCT

 ?fj_article
 ?wikidata
 ?wikidata_label
 ?DOI

WHERE {
  ?fj_article <http://www.w3.org/ns/prov-o#specializationOf> ?mesh_url .
  BIND(REPLACE(STR(?mesh_url), ".*MESH/D", "D") AS ?MESH) 
  service <https://query.wikidata.org/sparql> {
    ?wikidata wikidata_wdt:P486 ?MESH .
    ?wikidata rdfs:label ?wikidata_label .
    FILTER (LANG(?wikidata_label) = "ml")
  }
}
LIMIT 100
"""
csv_filename = 'fairjupyter_wikidata_match_articles_researchfield_malayalam'
results = query_and_display_results(query_string, csv_filename)
results

Results written to results/fairjupyter_wikidata_match_articles_researchfield_malayalam.csv


,fj_article,wikidata,wikidata_label,DOI
0,https://w3id.org/reproduceme/article_2746,http://www.wikidata.org/entity/Q7397,കമ്പ്യൂട്ടർ സോഫ്റ്റ്വെയർ,
1,https://w3id.org/reproduceme/article_1937,http://www.wikidata.org/entity/Q7397,കമ്പ്യൂട്ടർ സോഫ്റ്റ്വെയർ,
2,https://w3id.org/reproduceme/article_1308,http://www.wikidata.org/entity/Q7397,കമ്പ്യൂട്ടർ സോഫ്റ്റ്വെയർ,
3,https://w3id.org/reproduceme/article_2949,http://www.wikidata.org/entity/Q7397,കമ്പ്യൂട്ടർ സോഫ്റ്റ്വെയർ,
4,https://w3id.org/reproduceme/article_2886,http://www.wikidata.org/entity/Q7397,കമ്പ്യൂട്ടർ സോഫ്റ്റ്വെയർ,
...,...,...,...,...
95,https://w3id.org/reproduceme/article_3042,http://www.wikidata.org/entity/Q7397,കമ്പ്യൂട്ടർ സോഫ്റ്റ്വെയർ,
96,https://w3id.org/reproduceme/article_2271,http://www.wikidata.org/entity/Q7397,കമ്പ്യൂട്ടർ സോഫ്റ്റ്വെയർ,
97,https://w3id.org/reproduceme/article_1200,http://www.wikidata.org/entity/Q7397,കമ്പ്യൂട്ടർ സോഫ്റ്റ്വെയർ,
98,https://w3id.org/reproduceme/article_2358,http://www.wikidata.org/entity/Q7397,കമ്പ്യൂട്ടർ സോഫ്റ്റ്വെയർ,
